https://haystack.deepset.ai/tutorials/36_building_fallbacks_with_conditional_routing

if database have no relevant information -> use the web as a fallback data source for your RAG application

In [8]:
import os
openai_key = False
hf_token = False 
serper_key = False
with open("secrets") as file:
    for line in file.readlines():
        key,value = line.strip().split("=")
        if key == 'OPENAI_API_KEY':
            openai_key = True
            os.environ[key]=value
        elif key == 'HF_TOKEN':
            hf_token = True
            os.environ[key]=value
        elif key == 'SERPERDEV_API_KEY':
            serper_key = True
            os.environ[key]=value

assert openai_key, 'OPENAI_API_KEY not found'
assert hf_token, 'HF_TOKEN not found'
assert serper_key, 'SERPER_API_KEY not found'


In [4]:
from haystack.dataclasses import Document

In [5]:
data = [
    '''Munich, the vibrant capital of Bavaria in southern Germany, exudes a perfect blend of rich cultural
    heritage and modern urban sophistication. Nestled along the banks of the Isar River, Munich is renowned
    for its splendid architecture, including the iconic Neues Rathaus (New Town Hall) at Marienplatz and
    the grandeur of Nymphenburg Palace. The city is a haven for art enthusiasts, with world-class museums like the
    Alte Pinakothek housing masterpieces by renowned artists. Munich is also famous for its lively beer gardens, where
    locals and tourists gather to enjoy the city's famed beers and traditional Bavarian cuisine. The city's annual
    Oktoberfest celebration, the world's largest beer festival, attracts millions of visitors from around the globe.
    Beyond its cultural and culinary delights, Munich offers picturesque parks like the English Garden, providing a
    serene escape within the heart of the bustling metropolis. Visitors are charmed by Munich's warm hospitality,
    making it a must-visit destination for travelers seeking a taste of both old-world charm and contemporary allure.'''
]
documents = [Document(text) for text in data]


In [2]:
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator


2025-01-31 13:03:35.415028: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 13:03:35.422182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-31 13:03:35.430629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-31 13:03:35.433162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 13:03:35.439738: I tensorflow/core/platform/cpu_feature_guar

In [21]:
#Serper web search
from haystack.components.websearch.serper_dev import SerperDevWebSearch

prompt_websearch = [
    ChatMessage.from_user(
        '''
            Answer the following query given the documents retrieved from the web.
            Your answer shoud indicate that your answer was generated from websearch.

            Query: {{query}}
            Documents:
            {% for document in documents %}
            {{document.content}}
            {% endfor %}
        '''
    )
]

websearch = SerperDevWebSearch() 
prompt_builder_websearch = ChatPromptBuilder(template = prompt_websearch)
llm_for_websearch = OpenAIChatGenerator(model='gpt-4o-mini')

### Conditional router
component that handles data routing on specific conditions.

for each route, define:
- a condition,
- an output,
- an output_name
- an output_type

In [14]:
from haystack.components.routers import ConditionalRouter

In [15]:
routes = [
    {
        "condition":"{{'no_answer' in replies[0].text}}",
        "output":"{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition":"{{'no_answer' not in replies[0].text}}",
        "output":"{{replies[0].text}}",
        "output_name": "answer",
        "output_type": str,
    }
]

router = ConditionalRouter(routes)

In [ ]:
#Building pipeline
from haystack import Pipeline

pipeline = Pipeline()

# Prompt
prompt_template = [
    ChatMessage.from_user(
        """
                Answer the following query given the documents.
        If the answer is not contained within the documents reply with 'no_answer'
        Query: {{query}}
        Documents:
        {% for document in documents %}
        {{document.content}}
        {% endfor %}
        """
    )
]

prompt_builder = ChatPromptBuilder(template = prompt_template) 
llm = OpenAIChatGenerator(model = 'gpt-4o-mini')

# Prompt websearch
prompt_websearch = [
    ChatMessage.from_user(
        '''
            Answer the following query given the documents retrieved from the web.
            Your answer shoud indicate that your answer was generated from websearch.

            Query: {{query}}
            Documents:
            {% for document in documents %}
            {{document.content}}
            {% endfor %}
        '''
    )
]

#router
routes = [
    {
        "condition":"{{'no_answer' in replies[0].text}}",
        "output":"{{query}}",
        "output_name": "go_to_websearch",
        "output_type": str,
    },
    {
        "condition":"{{'no_answer' not in replies[0].text}}",
        "output":"{{replies[0].text}}",
        "output_name": "answer",
        "output_type": str,
    }
]

router = ConditionalRouter(routes)

websearch = SerperDevWebSearch() 
prompt_builder_websearch = ChatPromptBuilder(template = prompt_websearch)
llm_for_websearch = OpenAIChatGenerator(model='gpt-4o-mini')


pipeline.add_component('prompt_builder', prompt_builder)
pipeline.add_component('llm', llm)
pipeline.add_component('llm_for_websearch', llm_for_websearch)
pipeline.add_component('websearch', websearch)
pipeline.add_component('prompt_websearch', prompt_builder_websearch)
pipeline.add_component('router', router)


pipeline.connect('prompt_builder.prompt', 'llm.messages')
pipeline.connect('llm.replies', 'router.replies')
#router

#if no answer found, go_to_websearch output

# prompt_websearch is result from serper api
pipeline.connect('router.go_to_websearch', 'websearch.query')
pipeline.connect('router.go_to_websearch', 'prompt_websearch.query')

#connect websearch to prompt_websearch
pipeline.connect('websearch.documents', 'prompt_websearch.documents')
#if answer found
pipeline.connect('prompt_websearch', 'llm_for_websearch')

pipeline.draw('conditional_routing.png')


